<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240127-%E5%BE%97%E5%88%B03class%E6%A0%B7%E6%9C%AC%E7%9A%842023NDRE%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(3.10-9.10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [2]:
import os
import pandas as pd
import math

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/sample_3class")

In [14]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


In [15]:
dataset = filteredSentinel.select(['NDRE','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDRE')


# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [16]:
# 5天序列NDRE
interval = 5
increment = 'day'
start = '2023-03-10'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2023-09-11').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDRE(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('NDRE'))

fitteds2 = s2makeup.map(add_fittedNDRE)

HANTS = fitteds2.select('NDRE')

In [17]:
# NDRE时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE', '25_NDRE', '26_NDRE', '27_NDRE', '28_NDRE', '29_NDRE', '30_NDRE', '31_NDRE', '32_NDRE', '33_NDRE', '34_NDRE', '35_NDRE', '36_NDRE', '37_NDRE']


In [18]:
image_agrgt_clip = image_agrgt.clip(roi)

In [19]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id','landcover'],scale=10,tileScale=8)

In [20]:
cols = ['id', 'landcover', '0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE', '25_NDRE', '26_NDRE', '27_NDRE', '28_NDRE', '29_NDRE', '30_NDRE', '31_NDRE', '32_NDRE', '33_NDRE', '34_NDRE', '35_NDRE', '36_NDRE', '37_NDRE']
inds = range(samples_datesets.size().getInfo())

In [21]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,landcover,0_NDRE,1_NDRE,2_NDRE,3_NDRE,4_NDRE,5_NDRE,6_NDRE,7_NDRE,...,28_NDRE,29_NDRE,30_NDRE,31_NDRE,32_NDRE,33_NDRE,34_NDRE,35_NDRE,36_NDRE,37_NDRE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(40),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
id,52.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.000000,61.000000,62.000000,63.000000,...,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000,51.000000
landcover,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
0_NDRE,0.403751,0.508312,0.546572,0.507265,0.399385,0.357012,0.338285,0.446259,0.409065,0.398649,...,0.264088,0.297422,0.301744,0.363949,0.381603,0.330130,0.206799,0.314038,0.391141,0.200837
1_NDRE,0.406092,0.510761,0.552599,0.511811,0.403014,0.360536,0.342277,0.457917,0.416150,0.400042,...,0.267714,0.298984,0.304830,0.371355,0.389123,0.337310,0.211106,0.319903,0.400076,0.203514
2_NDRE,0.408476,0.512664,0.558180,0.516131,0.406894,0.364155,0.346693,0.469692,0.423485,0.401569,...,0.271655,0.301011,0.308029,0.378874,0.396838,0.344643,0.215933,0.325728,0.408895,0.206289


In [23]:
df.to_csv('sample_NDRE_2023_247.csv',encoding='utf-8')